In [ ]:
### IMPORT DATA ###

# Code from https://community.amstat.org/dataexpo/home

# Data dictionary reference https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ

import os

import urllib.request

from concurrent.futures import ThreadPoolExecutor


base_url = "https://blobs.duckdb.org/flight-data-partitioned/"

files = [f"Year={year}/data_0.parquet" for year in range(1987, 2025)]


def download_file(f):

    os.makedirs(os.path.dirname(f), exist_ok=True)

    req = urllib.request.Request(base_url + f, headers={'User-Agent': 'Mozilla/5.0'})

    with urllib.request.urlopen(req) as response, open(f, 'wb') as out_file:

        out_file.write(response.read())


with ThreadPoolExecutor() as executor:

    executor.map(download_file, files)

In [1]:
# Imports
import polars as pl
from sklearn.cluster import KMeans
import numpy as np

In [35]:
# Load in Data to a DataFrame
df_2024 = pl.read_parquet('Data/Year=2024/data_0.parquet')

In [36]:
# Head
print(df_2024.head(3))

shape: (3, 110)
┌──────┬─────────┬───────┬────────────┬───┬───────────────┬──────────────┬─────────────┬───────────┐
│ Year ┆ Quarter ┆ Month ┆ DayofMonth ┆ … ┆ Div5LongestGT ┆ Div5WheelsOf ┆ Div5TailNum ┆ column109 │
│ ---  ┆ ---     ┆ ---   ┆ ---        ┆   ┆ ime           ┆ f            ┆ ---         ┆ ---       │
│ i64  ┆ i64     ┆ i64   ┆ i64        ┆   ┆ ---           ┆ ---          ┆ str         ┆ str       │
│      ┆         ┆       ┆            ┆   ┆ str           ┆ str          ┆             ┆           │
╞══════╪═════════╪═══════╪════════════╪═══╪═══════════════╪══════════════╪═════════════╪═══════════╡
│ 2024 ┆ 1       ┆ 1     ┆ 8          ┆ … ┆ null          ┆ null         ┆ null        ┆ null      │
│ 2024 ┆ 1       ┆ 1     ┆ 9          ┆ … ┆ null          ┆ null         ┆ null        ┆ null      │
│ 2024 ┆ 1       ┆ 1     ┆ 10         ┆ … ┆ null          ┆ null         ┆ null        ┆ null      │
└──────┴─────────┴───────┴────────────┴───┴───────────────┴──────────────┴─

In [37]:
# Describe
print(df_2024.describe())

shape: (9, 111)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ Year      ┆ Quarter   ┆ Month     ┆ … ┆ Div5Longe ┆ Div5Wheel ┆ Div5TailN ┆ column10 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ stGTime   ┆ sOff      ┆ um        ┆ 9        │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 3.461319e ┆ 3.461319e ┆ 3.461319e ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0        │
│           ┆ 6         ┆ 6         ┆ 6         ┆   ┆           ┆           ┆           ┆          │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 3461319   ┆ 3461319   ┆ 3461319   ┆ 3461319  │
│ t         ┆           ┆           ┆           ┆   ┆           ┆          

In [38]:
# Schema
print(df_2024.schema)

Schema([('Year', Int64), ('Quarter', Int64), ('Month', Int64), ('DayofMonth', Int64), ('DayOfWeek', Int64), ('FlightDate', Date), ('Reporting_Airline', String), ('DOT_ID_Reporting_Airline', Int64), ('IATA_CODE_Reporting_Airline', String), ('Tail_Number', String), ('Flight_Number_Reporting_Airline', Int64), ('OriginAirportID', Int64), ('OriginAirportSeqID', Int64), ('OriginCityMarketID', Int64), ('Origin', String), ('OriginCityName', String), ('OriginState', String), ('OriginStateFips', String), ('OriginStateName', String), ('OriginWac', Int64), ('DestAirportID', Int64), ('DestAirportSeqID', Int64), ('DestCityMarketID', Int64), ('Dest', String), ('DestCityName', String), ('DestState', String), ('DestStateFips', String), ('DestStateName', String), ('DestWac', Int64), ('CRSDepTime', String), ('DepTime', String), ('DepDelay', Float64), ('DepDelayMinutes', Float64), ('DepDel15', Float64), ('DepartureDelayGroups', Int64), ('DepTimeBlk', String), ('TaxiOut', String), ('WheelsOff', String), ('

In [39]:
### Question 1 ------ Looking at fuel price by region and its correlation with x

#sub_set = df_2024.select(['Reporting_Airline', 'Cancelled', ''])

Question 2 ------ Can flights be grouped into clusters based on delay causes

In [40]:
### Subset needed Data ###
sub_set_2 = df_2024.select([
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
])

In [41]:
# Drop nulls
sub_set_2 = sub_set_2.drop_nulls()

In [42]:
# Head
print(sub_set_2.schema)
print(sub_set_2.head)

Schema([('CarrierDelay', String), ('WeatherDelay', String), ('NASDelay', String), ('SecurityDelay', String), ('LateAircraftDelay', String)])
<bound method DataFrame.head of shape: (748_647, 5)
┌──────────────┬──────────────┬──────────┬───────────────┬───────────────────┐
│ CarrierDelay ┆ WeatherDelay ┆ NASDelay ┆ SecurityDelay ┆ LateAircraftDelay │
│ ---          ┆ ---          ┆ ---      ┆ ---           ┆ ---               │
│ str          ┆ str          ┆ str      ┆ str           ┆ str               │
╞══════════════╪══════════════╪══════════╪═══════════════╪═══════════════════╡
│ 0.00         ┆ 0.00         ┆ 4.00     ┆ 0.00          ┆ 23.00             │
│ 27.00        ┆ 0.00         ┆ 4.00     ┆ 0.00          ┆ 0.00              │
│ 0.00         ┆ 0.00         ┆ 43.00    ┆ 0.00          ┆ 0.00              │
│ 0.00         ┆ 8.00         ┆ 22.00    ┆ 0.00          ┆ 2.00              │
│ 71.00        ┆ 0.00         ┆ 87.00    ┆ 0.00          ┆ 19.00             │
│ …            ┆ 

In [43]:
# Simple clustering model

# Fit the clustering model
kmeans = KMeans(n_clusters=4, random_state=42)
cluster = kmeans.fit_predict(sub_set_2)

# Numpy array
#cluster = np.array(cluster)

# Add to data
sub_set_2 = sub_set_2.with_columns(
    cluster=cluster
)

print(sub_set_2.head)




<bound method DataFrame.head of shape: (748_647, 6)
┌──────────────┬──────────────┬──────────┬───────────────┬───────────────────┬─────────┐
│ CarrierDelay ┆ WeatherDelay ┆ NASDelay ┆ SecurityDelay ┆ LateAircraftDelay ┆ cluster │
│ ---          ┆ ---          ┆ ---      ┆ ---           ┆ ---               ┆ ---     │
│ str          ┆ str          ┆ str      ┆ str           ┆ str               ┆ i32     │
╞══════════════╪══════════════╪══════════╪═══════════════╪═══════════════════╪═════════╡
│ 0.00         ┆ 0.00         ┆ 4.00     ┆ 0.00          ┆ 23.00             ┆ 0       │
│ 27.00        ┆ 0.00         ┆ 4.00     ┆ 0.00          ┆ 0.00              ┆ 0       │
│ 0.00         ┆ 0.00         ┆ 43.00    ┆ 0.00          ┆ 0.00              ┆ 0       │
│ 0.00         ┆ 8.00         ┆ 22.00    ┆ 0.00          ┆ 2.00              ┆ 0       │
│ 71.00        ┆ 0.00         ┆ 87.00    ┆ 0.00          ┆ 19.00             ┆ 0       │
│ …            ┆ …            ┆ …        ┆ …             ┆

In [44]:
# Analyze the centers
print(kmeans.cluster_centers_)

[[ 1.16613565e+01  4.97735777e+00  1.52680179e+01  1.54639520e-01
   1.78918948e+01]
 [ 1.70919690e+02  4.91291353e-01  5.16469133e+00  2.11274863e-02
   7.39799547e+00]
 [ 8.62250594e+02  5.28206651e-01  6.14192399e+00 -1.66533454e-16
   3.96552850e+01]
 [ 9.65730913e+00  4.65735011e+00  6.53280603e+00  7.28863571e-02
   2.03542867e+02]]


In [45]:
# Group by cluster to count how many of each cluster there is

cluster_counts = sub_set_2.group_by("cluster").len(name="n")

print(cluster_counts)


shape: (4, 2)
┌─────────┬────────┐
│ cluster ┆ n      │
│ ---     ┆ ---    │
│ i32     ┆ u32    │
╞═════════╪════════╡
│ 2       ┆ 3368   │
│ 3       ┆ 48803  │
│ 0       ┆ 657672 │
│ 1       ┆ 38804  │
└─────────┴────────┘


In [46]:
### Question 3 ------- Are there seasonal trends to flight cancellations (Seasonal decomposition)

In [47]:
### Question 4 ------- What factors are most predictive for flight diversions?